In [1]:
import os, csv
from csv import writer, DictWriter
from os import listdir
import os.path as osp
from os.path import isfile, isdir, join

import newspaper
from newspaper import Config, Article, Source
from newspaper.article import ArticleException as aeN
import time
import datetime
from datetime import datetime as DT
import json
# setup widget import
from ipywidgets import widgets
from IPython.display import display

import nltk
# nltk.download('punkt')
# Locals 
from utils import *

In [2]:
popsdict = fetchPopularList(override='popular-with-articles3.json', witharticles=True)
pops = popsdict['Popular_Articles']
print (len(pops))

 Using JSON from popular-with-articles3.json
57


In [3]:
def validateSources(pops):
    titles = []
    progress_bar = widgets.IntProgress(min=0, max=len(pops), description="Extracting")
    display(progress_bar)

    for url in pops:
        print(validateSource(url))
        progress_bar.value += 1
    
    progress_bar.bar_style = "success"
    progress_bar.close()


In [4]:
validateSources(pops[0:100])
print ('PARTIALLY DONE!', end="\n\n")

validateSources(pops[100:])
print ('DONE DONE!')


IntProgress(value=0, description='Extracting', max=57)

News URL: https://abcnews.go.com/Technology Success 200. Articles available: 195
News URL: https://ai.googleblog.com/ Success 200. Articles available: 11
News URL: https://arstechnica.com/gadgets/ Success 200. Articles available: 457
News URL: https://blogs.cisco.com/innovation Success 200. Articles available: 24
News URL: https://www.cnn.com/specials/tech/innovate Success 200. Articles available: 732
News URL: https://www.discovermagazine.com/technology Success 200. Articles available: 89
News URL: https://www.discovery.com/technology Success 200. Articles available: 61
News URL: https://federalnewsnetwork.com/category/technology-main/cybersecurity/ Success 200. Articles available: 51
News URL: https://federalnewsnetwork.com/category/technology-main/cloud-computing/ Success 200. Articles available: 51
News URL: https://federalnewsnetwork.com/category/technology-main/it-modernization/ Success 200. Articles available: 51
News URL: https://governmenttechnologyinsider.com/category/digital

IntProgress(value=0, description='Extracting', max=0)

DONE DONE!


In [5]:
### test a random URL

# url = "https://www.fda.gov/about-fda/innovation-fda/innovation-news"
# url = "https://www.irs.gov/a-closer-look"
# url = "https://www.futurist.com/blog"
# url = "https://ourpublicservice.org/our-solutions/innovation-and-technology-modernization/innovation/"
# url = "https://ourpublicservice.org/"
# url = "https://medium.com/"
# url = "https://federalnewsnetwork.com"
# print(validateSource(url))
# url = "https://governmentciomedia.com"
# print(validateSource(url))
# url = "https://fcw.com"
# print(validateSource(url))
# url = "https://nextgov.com"
# print(validateSource(url))
# url = "https://www.lastweekinaws.com/blog/?_categories=aws"
url = "https://money.cnn.com/2016/09/23/news/economy/white-working-class-economy/index.html"
print(validateSource(url))

News URL: https://money.cnn.com/2016/09/23/news/economy/white-working-class-economy/index.html Success 200. Articles available: 11


In [ ]:
import random
randset = random.sample(pops, 10) 

In [ ]:
# additional 'interesting validations'
## Check a specific site versus keywords


In [10]:
# site = 'https://www.independent.co.uk/arts-entertainment/music/news/gulsen-bayraktar-colakoglu-arrested-turkey-b2153305.html'
site = "https://money.cnn.com/2016/09/23/news/economy/white-working-class-economy/index.html"
title = ""
summary = ""
keywords = []
top_image = ""
authors = []
url = ""
publish_date = ""
an_article = Article(url=site)
an_article.download()
an_article.throw_if_not_downloaded_verbose()
an_article.parse()
an_article.nlp()
print(f"article.url: {site} ")
print(f"publish date: {an_article.publish_date}")

title = an_article.title.strip()
summary = an_article.summary
keywords = an_article.keywords
authors = an_article.authors
top_image = an_article.top_image

print(f"title: {title}")
print(f"summary: {summary}")
print(f"keywords: {keywords}")
print(f"top_image: {top_image}")


article.url: https://money.cnn.com/2016/09/23/news/economy/white-working-class-economy/index.html 
publish date: 2016-09-23 00:00:00
title: The economy stinks, but I'm doing OK, say working class whites
summary: Those are some of the reasons why working class whites feel the economy is headed in the wrong direction.
Nearly two-thirds of working class whites say they are satisfied with their own personal financial situation.
Related: Working class whites blame Washington, but still want more government helpHis four daughters will have to deal with all of this.
Working class whites were less likely to get a boost in pay than their college educated peers.
White, Working Class & Worried is a CNN partnership with the Kaiser Family Foundation - an extensive survey of white, working class Americans and voters who form a backbone of the support for Donald Trump.
keywords: ['class', 'whites', 'economic', 'feel', 'security', 'working', 'ok', 'im', 'economy', 'doing', 'say', 'state', 'stinks', 'w

In [ ]:
TOPICS_TABLE_NAME  = "InnovationBlog_Article_Topics"
ARTICLES_TABLE_NAME= "innovation-articles"
from APIutils import *

# can we open a connection to the data base?
ddb = getDBConnection()

## test evaluate versus terms
cumulativestrength = 0
for keyword in keywords:
    strength = evaluateAgainstKeyTerms(keyword)
    if strength:
        print(f"term: {keyword} term strength: {strength}")
        cumulativestrength += 1
print(f"Title     : {title} Cumulative Strength: {cumulativestrength}" )

if cumulativestrength:
    # it is possible that other terms in keywords might be 'undiscovered' indicative words as well
    print(f"Possible other keywords that might controbute?: {keywords}" )
